In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

In [2]:
from fileIO import fileIO
from fileInfo import fileInfo
from pandas import read_csv, DataFrame, Series
from fsUtils import setDir, setFile
from timeUtils import timestat
from numpy import nan
from pandas import to_datetime, NaT, isna, concat

from glob import glob
from fileIO import csvIO, pickleIO


def loadData(ifile):
    io = csvIO()
    mbdata = io.get(ifile, delimiter="\t", header=None)
    #mbdata = read_csv(ifile, delimiter="\t", header=None)
    mbdata = mbdata.replace('\\N', nan)
    return mbdata


def getData(files, colnames):
    data = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
    print("Keys: {0}".format(data.keys()))
    data = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in data.items() if key in colnames} if colnames is not None else data
    print("Keys: {0}".format(data.keys()))
    return data


def setIndex(data):
    for key,df in data.items():
        colname = df.columns[0]
        df.index = df[colname]
        df.drop([colname], axis=1, inplace=True)
    return data


def createDate(year, month, day):
    if all([isinstance(x,str) for x in [year,month,day]]):
        return to_datetime('{0}-{1}-{2}'.format(year, month, day), format='%Y-%m-%d', errors='ignore')
    elif all([isinstance(x,str) for x in [year,month]]):
        return to_datetime('{0}-{1}'.format(year, month), format='%Y-%m', errors='ignore')
    elif all([isinstance(x,str) for x in [year]]):
        return to_datetime('{0}'.format(year), format='%Y', errors='ignore')
    return NaT


def convertToDatetime(year, month, day):
    year  = year.apply(lambda x: int(x) if (not isna(x) and x.isdigit()) else -1)
    month = month.apply(lambda x: int(x) if (not isna(x) and x.isdigit()) else -1)
    day   = day.apply(lambda x: int(x) if (not isna(x) and x.isdigit()) else -1)
    tmp   = DataFrame(year).join(month).join(day)
    tmp.columns = ["year", "month", "day"]
    return to_datetime(tmp, errors='coerce')


basedir = "./"
basedir = "/Volumes/Seagate/DB"

aIDs={"ArianaGrande": 823336, "BuddyHolly": 10937, "Rupaul": 34318, "U2": 197, "DMB": 502, "Bono": 35575}

# DB Data

In [ ]:
lengthData = open("flength.csv").readlines()
lengthData = [x.replace("\n", "").strip().split() for x in lengthData]
lengthData = Series({item[1].split("/")[1]: int(item[0]) for item in lengthData if len(item[1].split("/")) > 1})
lengthData

In [ ]:
list(lengthData.index)

In [ ]:
#lengthData['release_label']
lengthData[(lengthData <= 6)]

# Lookup Dictionary

In [ ]:
colnames = {}
colnames['gender'] = {0: "GenderID", 1: "GenderName"} #, 3: "NA3", "GenderDescr"}

ts = timestat("Loading Gender Data")
files = [ifile for ifile in glob("mbdump/gender*")]
genderData = getData(files, colnames)
genderData = setIndex(genderData)
ts.stop()

dGenderName = genderData['gender']['GenderName'].to_dict()
del genderData

In [ ]:
colnames = {}
colnames["area_type"] = {0: "AreaTypeID", 1: "AreaTypeName", 3: "AlsoAreaTypeID", 4: "AreaTypeDescr", 5: "AreaTypeGID"}
colnames["area"]      = {0: "AreaID", 1: "AreaGID", 2: "AreaName", 3: "AreaTypeID"}
#colnames["area_gid_redirect"] = {0: "AreaGIDUUID", 1: "AreaGID"}
#colnames["area_alias_type"] = {0: "AreaAliasTypeID", 1: "AreaAliasTypeName", 5: "AreaAliasTypeUUID"}
#colnames["area_alias"] = {0: "AreaAliasID", 1: "NA1", 2: "AreaAlias", 3: "AreaLang", 6: "AliasTypeID", 7: "AreaSortName"}

ts = timestat("Loading Area Data")
files = glob("mbdump/area*")
areaData = getData(files, colnames)
areaData = setIndex(areaData)
ts.stop()

dAreaName = areaData['area']['AreaName'].to_dict()
del areaData

In [ ]:
colnames = {}
colnames['isrc'] = {0: "ISRCID", 1: "RecordingID", 2: "ISRC"}
colnames['iswc'] = {0: "ISWCID", 1: "WorkID", 2: "ISWC"}
colnames['iso_3166_1'] = {0: "ISO31661ID", 1: "ISO31661"}
colnames['iso_3166_2'] = {0: "ISO31662ID", 1: "ISO31662"}
colnames['iso_3166_3'] = {0: "ISO31663ID", 1: "ISO31663"}

ts = timestat("Loading i* Code Data")
files = glob("mbdump/is*")
icodeData = getData(files, colnames)
icodeData = setIndex(icodeData)
ts.stop()

iSWCData = icodeData['iswc']["ISWC"].copy(deep=True)
iSWCData.index = icodeData['iswc']['WorkID']
iSWCData = iSWCData.drop_duplicates()
del icodeData

In [ ]:
colnames = {}
colnames["language"] = {0: "LanguageID", 1: "LanguageShort1", 2: "LanguageShort2", 3: "LanguageShort3", 4: "LanguageName", 5: "NA5", 6: "LanguageShort"}

ts = timestat("Loading URL Data")
files = glob("mbdump/language*")
languageData = getData(files, colnames)
languageData = setIndex(languageData)
ts.stop()

dLanguageName = languageData['language']['LanguageName'].to_dict()
del languageData

In [ ]:
colnames = {}
colnames["script"] = {0: "ScriptID", 1: "ScriptName", 2: "NA2", 3: "ScriptDescr", 4: "NA4"}

ts = timestat("Loading URL Data")
files = glob("mbdump/script")
scriptData = getData(files, colnames)
scriptData = setIndex(scriptData)
ts.stop()

dScriptName = scriptData['script']['ScriptName'].to_dict()
del scriptData

In [ ]:
colnames = {}
colnames["label_alias"] = {0: "LabelAliasID", 1: "LabelID", 2: "LabelAliasName", 7: "LabelAliasName2"}
colnames["label_alias_type"] = {0: "LabelAliasTypeID", 1: "LabelAliasTypeName", 5: "LabelAliasGID"}
colnames["label_ipi"] = {0: "LabelIPIID", 1: "LabelIPI"}
colnames["label_isni"] = {0: "LabelISNIID", 1: "LabelISNI"}
colnames["label_type"] = {0: "LabelTypeID", 1: "LabelTypeName", 5: "LabelTypeGID"}
colnames["label"] = {0: "LabelID", 1: "LabelGID", 2: "LabelName"}

ts = timestat("Loading Label Data")
files = glob("mbdump/label*")
labelData = getData(files, colnames)
labelData = setIndex(labelData)
ts.stop()

dLabelName = labelData['label']['LabelName'].to_dict()
del labelData

# Gender

In [ ]:
colnames = {}
colnames['gender'] = {0: "GenderID", 1: "GenderName"} #, 3: "NA3", "GenderDescr"}

ts = timestat("Loading Gender Data")
files = [ifile for ifile in glob("mbdump/gender*")]
genderData = getData(files, colnames)
genderData = setIndex(genderData)
ts.stop()

# Lookup

In [ ]:
colnames = {}
colnames["l_label_release"] = {0: "Index", 1: "NA1", 2: "NA2", 3: "ReleaseID"}

ts = timestat("Loading Area Data")
files = glob("mbdump/l_label_release")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
lookupData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in lookupData.items()} if colnames is not None else lookupData
print("Keys: {0}".format(lookupData.keys()))
ts.stop()

In [ ]:
lookupData['l_label_release'][lookupData['l_label_release']['ReleaseID'].isin(releaseIDs)]

In [ ]:
lookupData['l_label_release'].max()

# Area

In [ ]:
colnames = {}
colnames["area_type"] = {0: "AreaTypeID", 1: "AreaTypeName", 3: "AlsoAreaTypeID", 4: "AreaTypeDescr", 5: "AreaTypeGID"}
colnames["area"]      = {0: "AreaID", 1: "AreaGID", 2: "AreaName", 3: "AreaTypeID"}
#colnames["area_gid_redirect"] = {0: "AreaGIDUUID", 1: "AreaGID"}
#colnames["area_alias_type"] = {0: "AreaAliasTypeID", 1: "AreaAliasTypeName", 5: "AreaAliasTypeUUID"}
#colnames["area_alias"] = {0: "AreaAliasID", 1: "NA1", 2: "AreaAlias", 3: "AreaLang", 6: "AliasTypeID", 7: "AreaSortName"}

ts = timestat("Loading Area Data")
files = glob("mbdump/area*")
areaData = getData(files, colnames)
areaData = setIndex(areaData)
ts.stop()

# Event

In [ ]:
colnames = {}
colnames["event_type"] = {0: "EventTypeID", 1: "EventTypeName", 4: "EventTypeDescr"}
colnames["event_alias_type"] = {0: "EventAliasTypeID", 1: "EventAliasTypeName", 5: "EventAliasTypeGID"}
colnames["event_alias"] = {0: "EventAliasID", 1: "EventID", 2: "EventAliasName", 3: "EventAliasLang", 7: "EventAliasName2"}
colnames["event"] = {0: "EventID", 1: "EventGID", 2: "EventName", 
                     3: "EventStartYear", 4: "EventStartMonth", 5: "EventStartDay", 6: "EventEndYear", 7: "EventEndMonth", 8: "EventEndDay"}

ts = timestat("Loading Event Data")
files = glob("mbdump/event*")
eventData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(eventData.keys()))
eventData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in eventData.items() if key in colnames} if colnames is not None else eventData
print("Keys: {0}".format(eventData.keys()))
ts.stop()

In [ ]:
eventData['event']

# ICode

In [ ]:
colnames = {}
colnames['isrc'] = {0: "ISRCID", 1: "RecordingID", 2: "ISRC"}
colnames['iswc'] = {0: "ISWCID", 1: "WorkID", 2: "ISWC"}
colnames['iso_3166_1'] = {0: "ISO31661ID", 1: "ISO31661"}
colnames['iso_3166_2'] = {0: "ISO31662ID", 1: "ISO31662"}
colnames['iso_3166_3'] = {0: "ISO31663ID", 1: "ISO31663"}

ts = timestat("Loading i* Code Data")
files = glob("mbdump/is*")
icodeData = getData(files, colnames)
icodeData = setIndex(icodeData)
ts.stop()

iSWCData = icodeData['iswc']["ISWC"].copy(deep=True)
iSWCData.index = icodeData['iswc']['WorkID']
iSWCData = iSWCData.drop_duplicates()
del icodeData

# Medium

In [ ]:
colnames = {}
colnames["medium_format"] = {0: "MediumFormatID", 1: "MediumName", 2: "MediumGroupID", 3: "NA3", 4: "MediumIntroYear", 5: "MediumDescr", 6: "MediumGID"}
#colnames["medium_cdtoc"]  = {0: "NA0", 1: "NA1", 2: "NA2", 3: "NA3"}
colnames["medium"] = {0: "ReleaseID_1", 1: "ReleaseID", 2: "SideNum", 3: "MediumFormatID", 4: "NA4", 5: "NA5", 7: "NumTracks"}

ts = timestat("Loading Medium Data")
files = glob("mbdump/medium*")
mediumData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(mediumData.keys()))
mediumData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in mediumData.items() if key in colnames} if colnames is not None else mediumData
print("Keys: {0}".format(mediumData.keys()))
ts.stop()

In [ ]:
mediumData['medium_format'].nunique()

In [ ]:
mediumData['medium'].nunique()

In [ ]:
## 12" Vinyl ; ReleaseID=1310220
#mediumData['medium'][mediumData['medium'].eq(1310220).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1310220   1274390    1       33        4
# 1351220   1310220    1       31        11      <- Matches https://musicbrainz.org/release/6b5f33a8-fc5e-4c1e-b379-c659ce20a1c8


## Digital Media : ReleaseID=1792210
#mediumData['medium'][mediumData['medium'].eq(1792210).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1792210   1694923    1      12	NaN	0	2015-11-30 00:32:22.335038+00	13
# 1904010   1792210    1      12	NaN	0	2016-06-20 18:41:01.106192+00	11   Both Match


# 2x12" Vinyl ; ReleaseID=1680415

#mediumData['medium'][mediumData['medium'].eq(1680415).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1680415	1598741	1	1	NaN	0	2015-04-27 04:37:49.277705+00	11
# 1775088	1680415	1	31	NaN	0	2015-10-31 12:02:35.843886+00	5
# 1775089	1680415	2	31	NaN	0	2015-10-31 12:02:35.843886+00	6

# 1792210   1694923    1      12	NaN	0	2015-11-30 00:32:22.335038+00	13
# 1904010   1792210    1      12	NaN	0	2016-06-20 18:41:01.106192+00	11   Both Match

# Pixies Velouria CD w/ 4 Tracks
# 3099, 1162482, 3097, 2259927

## CD
# mediumData['medium'][mediumData['medium'].eq(3099).any(1)]
# 3099	3099	1	1	NaN	0	2011-05-16 14:57:06.530063+00	4

## CD (Status=Promotional)
# mediumData['medium'][mediumData['medium'].eq(1162482).any(1)]
# 1162482	1146184	1	12	NaN	0	2012-04-14 05:35:56.931961+00	1
# 1181517	1162482	1	1	NaN	0	2012-05-30 00:53:24.512335+00	4

## CD
# mediumData['medium'][mediumData['medium'].eq(3097).any(1)]
# 3097	3097	1	1	NaN	0	2012-10-18 19:49:17.567219+00	4

## 12" Vinyl
# mediumData['medium'][mediumData['medium'].eq(2259927).any(1)]
# 2259927	2099781	1	12	NaN	0	2018-01-11 08:03:13.107915+00	5
# 2441117	2259927	1	31	NaN	0	2018-09-28 23:20:29.279923+00	4  <-- This matches Web

In [ ]:
mediumData['medium'].nunique()

In [ ]:
mediumData['medium'].shape

In [ ]:
mediumData['medium_cdtoc'][mediumData['medium_cdtoc'].eq(1598741).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(38).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq('38').any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(63).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(64).any(1)]

# Language

In [ ]:
colnames = {}
colnames["language"] = {0: "LanguageID", 1: "LanguageShort1", 2: "LanguageShort2", 3: "LanguageShort3", 4: "LanguageName", 5: "NA5", 6: "LanguageShort"}

ts = timestat("Loading URL Data")
files = glob("mbdump/language*")
languageData = getData(files, colnames)
languageData = setIndex(languageData)
ts.stop()

# Script

In [ ]:
colnames = {}
colnames["script"] = {0: "ScriptID", 1: "ScriptName", 2: "NA2", 3: "ScriptDescr", 4: "NA4"}

ts = timestat("Loading URL Data")
files = glob("mbdump/script")
scriptData = getData(files, colnames)
scriptData = setIndex(scriptData)
ts.stop()

# URL

In [ ]:
colnames = {}
colnames["url"] = {0: "URLID", 1: "URLGID", 2: "URLName"}
#colnames["url_gid_redirect"] = {0: "URLGIDUUID", 1: "URLGIDID"}

ts = timestat("Loading URL Data")
files = glob("mbdump/url*")
urlData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(urlData.keys()))
urlData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in urlData.items() if key in colnames} if colnames is not None else urlData
print("Keys: {0}".format(urlData.keys()))
ts.stop()

# Work

In [ ]:
colnames = {}
colnames["work_type"] = {0: "WorkTypeID", 1: "WorkTypeName", 3: "WorkTypeRanking", 4: "WorkTypeDescr", 5: "WorkTypeGID"}
#colnames["work_alias"] = {0: "WorkAliasID", 1: "WorkID", 2: "WorkName", 3: "WorkLang", 7: "WorkName2"}
#colnames["work_alias_type"] = {0: "WorkAliasTypeID", 1: "WorkAliasTypeName", 5: "WorkAliasTypeGID"}
colnames['work_attribute_type_allowed_value'] = {0: 'WorkAttributeTypeValueID', 1: "WorkAttributeTypeID", 2: "WorkAttributeTypeValue", 6: "WorkAttributeTypeValueGID"}
colnames["work_attribute_type"] = {0: "WorkAttributeTypeID", 1: "WorkAttributeTypeName", 6: "WorkAttributeTypeDescr"}
colnames["work_attribute"] = {0: "WorkAttributeID", 1: "WorkID", 2: "WorkAttributeTypeID", 3: "WorkAttributeTypeValueID", 4: "WorkAttributeCode"}
colnames["work_language"] = {0: "WorkID", 1: "LanguageID"}
colnames["work"] = {0: "WorkID", 1: "WorkGID", 2: "WorkName", 3: "WorkTypeID"} #, 4: "WorkDescr"}

ts = timestat("Loading Work Data")
files = glob("mbdump/work*")
workData = getData(files, colnames)
workData = setIndex(workData)
ts.stop()

In [ ]:
files = glob("mbdump/work_attribute_type_allowed_value")
tmpData = getData(files, None)

In [ ]:
workData['work_attribute']

In [ ]:
workData['work_attribute_type_allowed_value']

In [ ]:
workData['work_attribute_type_allowed_value'][workData['work_attribute_type_allowed_value']['NA1'] == 31]

In [ ]:
workData['work_attribute_type_allowed_value']['NA1'].value_counts()
"""
NA1
15    304
4     244
32    203
17    105
16     83
1      56
31     27
5      19
"""

In [ ]:
workData['work_attribute'][workData['work_attribute']['WorkAttributeTypeValueID'] == '443']

In [ ]:
workData['work_attribute'][workData['work_attribute']['WorkID'] == 13125643]

## Append Data And Create Master Work DataFrame

In [ ]:
ts = timestat("Joining Release Language Name")
dLanguageName = languageData['language']['LanguageName'].to_dict()
workData['work_language']["Language"] = workData['work_language']['LanguageID'].apply(lambda x: dLanguageName.get(x) if not isna(x) else None)
workData['work_language'].drop(["LanguageID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Work Attribute Type")
dWorkAttributeTypeName = workData['work_attribute_type']["WorkAttributeTypeName"].to_dict()
workData['work_attribute_type_allowed_value']["WorkAttributeType"] = workData['work_attribute_type_allowed_value']['WorkAttributeTypeID'].apply(lambda x: dWorkAttributeTypeName.get(x) if not isna(x) else None)
workData['work_attribute_type_allowed_value'].drop(["WorkAttributeTypeID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Work Attribute Type")
dWorkAttributeTypeName = workData['work_attribute_type']["WorkAttributeTypeName"].to_dict()
workData['work_attribute']["WorkAttributeType"] = workData['work_attribute']['WorkAttributeTypeID'].apply(lambda x: dWorkAttributeTypeName.get(x) if not isna(x) else None)
workData['work_attribute'].drop(["WorkAttributeTypeID"], axis=1, inplace=True)
ts.stop()

if False:
    ts = timestat("Joining Work Type")
    dWorkTypeName = workData['work_type']["WorkTypeName"].to_dict()
    workData['work']["WorkType"] = workData['work']['WorkTypeID'].apply(lambda x: dWorkTypeName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
    workData['work'].drop(["WorkTypeID"], axis=1, inplace=True)
    ts.stop()

    ts = timestat("Joining Work ISWC Data")
    iSWCData = icodeData['iswc']["ISWC"].copy(deep=True)
    iSWCData.index = icodeData['iswc']['WorkID']
    iSWCData = iSWCData.drop_duplicates()
    workData['work'] = workData['work'].join(iSWCData)
    ts.stop()

    ts = timestat("Grouping Attributes By WorkID")
    workAttributes = Series({workID: list(zip(df["WorkAttributeType"], df["WorkAttributeCode"])) for workID,df in workData['work_attribute'].groupby('WorkID')})
    workAttributes.name = "WorkAttributes"
    workData['work'] = workData['work'].join(workAttributes)
    ts.stop()


    savedir = setDir(basedir, "MusicBrainzData")
    savename = setFile(savedir, "{0}.p".format("workDataFrame"))
    ts = timestat("Saving Master Release DataFrame To {0}".format(savename))
    pickleIO().save(idata=workData['release'], ifile=savename)
    fsize = fileInfo(savename).fsize
    print("Master Release DataFrame is {0}{1}".format(fsize[0], fsize[1]))
    ts.stop()


In [ ]:
jData = workData['work_attribute_type_allowed_value'].reset_index()

In [ ]:
dWorkAttributeTypeValue = workData['work_attribute_type_allowed_value'][["WorkAttributeTypeValue", "WorkAttributeType"]].T.to_dict()

In [ ]:
noVal={"WorkAttributeTypeValue": None, "WorkAttributeType": None}
WorkAttributeTypeValue = workData['work_attribute']["WorkAttributeTypeValueID"].apply(lambda x: dWorkAttributeTypeValue.get(int(x)) if (not isna(x) and x.isdigit()) else noVal)

In [ ]:
WorkAttributeTypeValue.apply(Series)

In [ ]:
dWorkAttributeTypeValue

In [ ]:
workData['work'][workData['work']['WorkGID'] == 'b8ee567b-0f09-3345-b794-b6279594aa29']

In [ ]:
#{workID: }
df = workData['work_attribute'][workData['work_attribute'].eq(213928).any(1)].copy(deep=True)

In [ ]:
workData['work'][workData['work']["WorkGID"] == 'b8ee567b-0f09-3345-b794-b6279594aa29']

In [ ]:
for key,df in workData.items():
    print(key)
    print(df[df.eq(213928).any(1)])

# Label

In [ ]:
colnames = {}
colnames["label_alias"] = {0: "LabelAliasID", 1: "LabelID", 2: "LabelAliasName", 7: "LabelAliasName2"}
colnames["label_alias_type"] = {0: "LabelAliasTypeID", 1: "LabelAliasTypeName", 5: "LabelAliasGID"}
colnames["label_ipi"] = {0: "LabelIPIID", 1: "LabelIPI"}
colnames["label_isni"] = {0: "LabelISNIID", 1: "LabelISNI"}
colnames["label_type"] = {0: "LabelTypeID", 1: "LabelTypeName", 5: "LabelTypeGID"}
colnames["label"] = {0: "LabelID", 1: "LabelGID", 2: "LabelName"}

ts = timestat("Loading Label Data")
files = glob("mbdump/label*")
labelData = getData(files, colnames)
labelData = setIndex(labelData)
ts.stop()

# Recording

In [ ]:
colnames = {}
colnames["recording_alias_type"] = {0: "RecordingAliasTypeID", 1: "RecordingAliasTypeName"}
colnames["recording_alias"] = {0: "RecordingAliasID", 1: "RecordingID", 2: "RecordingAliasName", 3: "RecordingAliasLang", 7: "recordingAliasName2"}
colnames["recording"] = {0: "RecordingID", 1: "RecordingGID", 2: "RecordingName", 3: "ArtistID", 4: "TimeLength"} #, 5: "RecordingDescr"}

ts = timestat("Loading Recording Data")
files = glob("mbdump/rec")
recordingData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(recordingData.keys()))
recordingData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in recordingData.items() if key in colnames} if colnames is not None else recordingData
print("Keys: {0}".format(recordingData.keys()))
ts.stop()

# Track

In [ ]:
colnames = {}
colnames["track"] = {0: "TrackID", 1: "TrackGID", 2: "RecordingID", 3: "NA3", 4: "TrackNum", 5: "TrackNumName", 6: "TrackName", 7: "ArtistID", 8: "TimeLength"}

# Release
# 2373946	7c5d14b4-cf40-4eb1-89e6-d448125d94f3	1987-12-12: Hampton Coliseum, Hampton, VA, USA	197	2128729	3	\N	120	28	\N		0	-1	2019-03-15 09:46:50.446876+00
            
# Release Group
# 2128729	f979a1c6-b6c2-4aad-b5b6-709c6c216752	1987-12-12: Hampton Coliseum, Hampton, VA, USA	197	1		0	2019-03-15 09:46:45.458509+00

# Recording
# 24365864	6a355a78-06c0-4e06-a15e-05d6e533255e	Sunday Bloody Sunday	197	366000		0	2019-03-15 12:29:22.049215+00	f

# Track
# 27710495	fc969b10-fdba-4b0c-82d9-aa6a7179e41f	24365864	2571008	7	7	Sunday Bloody Sunday	197	366000	0	2019-03-15 12:29:22.049215+00	f
ts = timestat("Loading Recording Data")
files = glob("mbdump/track")
trackData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(trackData.keys()))
trackData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in trackData.items() if key in colnames} if colnames is not None else trackData
print("Keys: {0}".format(trackData.keys()))
ts.stop()

# Release

In [ ]:
colnames = {}
#colnames["release_alias_type"] = {0: "ReleaseAliasTypeID", 1: "ReleaseAliasTypeName"}
#colnames["release_alias"] = {0: "ReleaseAliasID", 1: "ReleaseID", 2: "ReleaseAliasName", 3: "ReleaseAliasLang", 7: "ReleaseAliasName2"}
colnames["release_status"] = {0: "ReleaseStatusID", 1: "ReleaseStatusName", 3: "NA3", 3: "ReleaseStatusDescr", 4: "ReleaseStatusGID"}
colnames["release_packaging"] = {0: "ReleasePackagingID", 1: "ReleasePackagingName", 3: "NA3", 4: "ReleasePackagingDescr", 5: "ReleaseStatusGID"}
colnames["release_label"] = {0: "Index", 1: "ReleaseID", 2: "LabelID", 3: "CatalogNumber"}
colnames["release_country"] = {0: "ReleaseID", 1: "ReleaseCountryID", 2: "ReleaseCountryYear", 3: "ReleaseCountryMonth", 4: "ReleaseCountryDay"}
colnames["release_unknown_country"] = {0: "ReleaseID", 1: "ReleaseCountryYear", 2: "ReleaseCountryMonth", 3: "ReleaseCountryDay"}
colnames["release"] = {0: "ReleaseID", 1: "ReleaseGID", 2: "ReleaseName", 3: "ArtistID", 4: "ReleaseGroupID", 5: "ReleaseStatusID", 
                       6: "ReleasePackagingID", 7: "LanguageID", 8: "ScriptID", 9: "ReleaseBarcode", 10: "ReleaseComment", 11: "NA11", 12: "NA12"}

ts = timestat("Loading Release Data")
files = [ifile for ifile in glob("mbdump/release*") if "group" not in ifile]
releaseData = getData(files, colnames)
releaseData = setIndex(releaseData)
ts.stop()

## Append Data And Create Master Release DataFrame

In [ ]:
tsRelease = timestat("Appending Release Data")

ts = timestat("Creating Release Country DateTime For {0} Releases".format(releaseData['release_country'].shape[0]))
tmp = releaseData['release_country'][["ReleaseCountryYear", "ReleaseCountryMonth", "ReleaseCountryDay"]]
tmp.columns = ["year", "month", "day"]
releaseData['release_country']['ReleaseDate'] = to_datetime(tmp, errors='ignore')
releaseData['release_country'].drop(["ReleaseCountryYear", "ReleaseCountryMonth", "ReleaseCountryDay"], axis=1, inplace=True)
ts.stop()

ts = timestat("Creating Release Unknown Country DateTime For {0} Releases".format(releaseData['release_unknown_country'].shape[0]))
tmp = releaseData['release_unknown_country'][["ReleaseCountryYear", "ReleaseCountryMonth", "ReleaseCountryDay"]]
tmp.columns = ["year", "month", "day"]
releaseData['release_unknown_country']['ReleaseDate'] = to_datetime(tmp, errors='ignore')
releaseData['release_unknown_country'].drop(["ReleaseCountryYear", "ReleaseCountryMonth", "ReleaseCountryDay"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Country Area")
dAreaName = areaData['area']['AreaName'].to_dict()
releaseData['release_country']["Country"] = releaseData['release_country']['ReleaseCountryID'].apply(lambda x: dAreaName.get(x) if not isna(x) else None)
releaseData['release_country'].drop(["ReleaseCountryID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Release Packaging Name")
dReleasePackagingName = releaseData['release_packaging']['ReleasePackagingName'].to_dict()
releaseData['release']["Packaging"] = releaseData['release']['ReleasePackagingID'].apply(lambda x: dReleasePackagingName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseData['release'].drop(["ReleasePackagingID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Release Status Name")
dReleaseStatusName = releaseData['release_status']['ReleaseStatusName'].to_dict()
releaseData['release']["Status"] = releaseData['release']['ReleaseStatusID'].apply(lambda x: dReleaseStatusName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseData['release'].drop(["ReleaseStatusID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Release Language Name")
dLanguageName = languageData['language']['LanguageName'].to_dict()
releaseData['release']["Language"] = releaseData['release']['LanguageID'].apply(lambda x: dLanguageName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseData['release'].drop(["LanguageID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Release Script Name")
dScriptName = scriptData['script']['ScriptName'].to_dict()
releaseData['release']["Script"] = releaseData['release']['ScriptID'].apply(lambda x: dScriptName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseData['release'].drop(["ScriptID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Country/Release Date")
tmp = concat([releaseData['release_country'], releaseData['release_unknown_country']]).reset_index()
releaseIDDate = tmp.sort_values(by="ReleaseDate").drop_duplicates(subset="ReleaseID")
releaseIDDate.index = releaseIDDate['ReleaseID']
releaseIDDate.drop(["ReleaseID"], axis=1, inplace=True)
releaseData['release'] = releaseData['release'].join(releaseIDDate)
ts.stop()

ts = timestat("Joining Label Data")
dLabelName = labelData['label']['LabelName'].to_dict()
releaseData['release_label']["Label"] = releaseData['release_label']['LabelID'].apply(lambda x: dLabelName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseLabel = releaseData['release_label'].drop_duplicates(subset="ReleaseID")[["ReleaseID", "Label"]].copy(deep=True)
releaseLabel["Label"].index = releaseLabel["ReleaseID"]
releaseData['release'].join(releaseLabel)
ts.stop()

ts = timestat("Dropping Last Columns")
releaseData['release'].drop(["ReleaseComment", "ReleaseBarcode", "NA11", "NA12"], axis=1, inplace=True)
ts.stop()


savedir = setDir(basedir, "MusicBrainzData")
savename = setFile(savedir, "{0}.p".format("ReleaseDataFrame"))
ts = timestat("Saving Master Release DataFrame To {0}".format(savename))
pickleIO().save(idata=releaseData['release'], ifile=savename)
fsize = fileInfo(savename).fsize
print("Master Release DataFrame is {0}{1}".format(fsize[0], fsize[1]))
ts.stop()


tsRelease.stop()

# Release-Group

In [ ]:
colnames = {}
#colnames["release_group_alias_type"] = {0: "ReleaseGroupAliasTypeID", 1: "ReleaseGroupAliasTypeName"}
#colnames["release_group_alias"] = {0: "ReleaseGroupAliasID", 1: "NA1", 2: "ReleaseGroupAliasName", 3: "ReleaseGroupAliasLang", 7: "ReleaseGroupAliasName2"}
colnames["release_group_primary_type"] = {0: "ReleaseGroupPrimaryTypeID", 1: "ReleaseGroupPrimaryTypeName", 3: "NA3"}
colnames["release_group_secondary_type"] = {0: "ReleaseGroupSecondaryTypeID", 1: "ReleaseGroupSecondaryTypeName"}
colnames["release_group"] = {0: "ReleaseGroupID", 1: "ReleaseGroupGID", 2: "ReleaseGroupName", 3: "ArtistID", 4: "ReleaseGroupPrimaryTypeID", 5: "ReleaseGroupComment", 6: "NA6"}
colnames["release_group_secondary_type_join"] = {0: "ReleaseGroupID", 1: "ReleaseGroupSecondaryTypeID"}

ts = timestat("Loading Release Data")
files = [ifile for ifile in glob("mbdump/release*") if "group" in ifile]
releaseGroupData = getData(files, colnames)
releaseGroupData = setIndex(releaseGroupData)
ts.stop()

## Append Data And Create Master ReleaseGroup DataFrame

In [ ]:
tsReleaseGroup = timestat("Appending ReleaseGroup Data")

ts = timestat("Joining Secondary Type Names")
dReleaseGroupSecondaryTypeName = releaseGroupData['release_group_secondary_type']['ReleaseGroupSecondaryTypeName'].to_dict()
releaseGroupData['release_group_secondary_type_join']['ReleaseGroupSecondaryType'] = releaseGroupData['release_group_secondary_type_join']['ReleaseGroupSecondaryTypeID'].apply(lambda x: dReleaseGroupSecondaryTypeName.get(x) if not isna(x) else None)
releaseGroupData['release_group_secondary_type_join'].drop(["ReleaseGroupSecondaryTypeID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Primary Type Names")
dReleaseGroupPrimaryTypeName = releaseGroupData['release_group_primary_type']['ReleaseGroupPrimaryTypeName'].to_dict()
releaseGroupData['release_group']['ReleaseGroupPrimaryType'] = releaseGroupData['release_group']['ReleaseGroupPrimaryTypeID'].apply(lambda x: dReleaseGroupPrimaryTypeName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
releaseGroupData['release_group'].drop(["ReleaseGroupPrimaryTypeID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Release Group And Secondary Type Join Data")
releaseGroupData['release_group'] = releaseGroupData['release_group'].join(releaseGroupData['release_group_secondary_type_join'])
ts.stop()

ts = timestat("Dropping Last Columns")
releaseGroupData['release_group'].drop(["NA6", "ReleaseGroupComment"], axis=1, inplace=True)
ts.stop()


savedir = setDir(basedir, "MusicBrainzData")
savename = setFile(savedir, "{0}.p".format("ReleaseGroupDataFrame"))
ts = timestat("Saving Master ReleaseGroup DataFrame To {0} (~20 sec)".format(savename))
from fileIO import pickleIO
pickleIO().save(idata=releaseGroupData['release_group'], ifile=savename)
fsize = fileInfo(savename).fsize
print("Master ReleaseGroup DataFrame is {0}{1}".format(fsize[0], fsize[1]))
ts.stop()


tsReleaseGroup.stop()

del releaseGroupData

# Artist

In [ ]:
colnames = {}
colnames["artist_credit"] = {0: "ArtistCreditID", 1: "ArtistCreditName", 2: "ArtistCreditNum", 3: "NA3"}
colnames["artist_type"] = {0: "ArtistTypeID", 1: "ArtistTypeName", 2: "NA2", 3: "NA3", 4: "ArtistTypeDescr", 5: "ArtistTypeGID"}
colnames["artist_isni"] = {0: "ArtistID", 1: "ISNICode"}
colnames["artist_alias_type"] = {0: "ArtistAliasTypeID", 1: "ArtistAliasTypeName", 5: "ArtistAliasTypeGID"}
colnames["artist_alias"] = {0: "ArtistAliasID", 1: "ArtistID", 2: "ArtistAliasName", 3: "ArtistAliasLang", 7: "ArtistAliasSortName"}
colnames["artist"] = {0: "ArtistID", 1: "ArtistGID", 2: "ArtistName", 3: "ArtistSortName",
                      4: "FormedYear", 5: "FormedMonth", 6: "FormedDay", 
                      7: "DisbandedYear", 8: "DisbandedMonth", 9: "DisbandedDay", 
                      10: "ArtistTypeID", 11: "CountryAreaID", 12: "GenderID", 13: "ArtistDescr", 14: "NA14", 17: "FoundedInAreaID", 18: "DisbandedInAreaID"}

ts = timestat("Loading Artist Data")
files = [ifile for ifile in glob("mbdump/artist*") if ifile not in ["mbdump/artist_credit_name"]]
artistData = getData(files, colnames)
artistData = setIndex(artistData)
ts.stop()

## Append Data And Create Master Artist DataFrame

In [ ]:
tsArtist = timestat("Appending Artist Data")

ts = timestat("Creating Formed/Disbanded DateTime For {0} Artists (~7 sec)".format(artistData['artist'].shape[0]))
artistData['artist']['Formed']    = convertToDatetime(artistData['artist']["FormedYear"], artistData['artist']["FormedMonth"], artistData['artist']["FormedDay"])
artistData['artist']['Disbanded'] = convertToDatetime(artistData['artist']["DisbandedYear"], artistData['artist']["DisbandedMonth"], artistData['artist']["DisbandedDay"])
artistData['artist'].drop(["FormedYear", "FormedMonth", "FormedDay", "DisbandedYear", "DisbandedMonth", "DisbandedDay"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Artist Type")
dArtistTypeName = artistData['artist_type']["ArtistTypeName"].to_dict()
artistData['artist']["ArtistType"] = artistData['artist']['ArtistTypeID'].apply(lambda x: dArtistTypeName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
artistData['artist'].drop(["ArtistTypeID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Gender Type")
artistData['artist']['Gender'] = artistData['artist']['GenderID'].apply(lambda x: dGenderName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
artistData['artist'].drop(["GenderID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining Area Type")
artistData['artist']["Country"]     = artistData['artist']['CountryAreaID'].apply(lambda x: dAreaName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
artistData['artist']["FormedIn"]    = artistData['artist']['FoundedInAreaID'].apply(lambda x: dAreaName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
artistData['artist']["DisbandedIn"] = artistData['artist']['DisbandedInAreaID'].apply(lambda x: dAreaName.get(int(x)) if (not isna(x) and x.isdigit()) else None)
artistData['artist'].drop(["CountryAreaID", "FoundedInAreaID", "DisbandedInAreaID"], axis=1, inplace=True)
ts.stop()

ts = timestat("Joining ISNI")
artistData['artist'] = artistData['artist'].join(artistData['artist_isni'])
ts.stop()

ts = timestat("Collecting and Joining Artist Aliases (~40 sec)")
artistAliases = DataFrame(Series({artistID: df["ArtistAliasName"].to_list() for artistID,df in artistData['artist_alias'].groupby("ArtistID")}))
artistAliases.columns = ["Aliases"]
artistData['artist'] = artistData['artist'].join(artistAliases)
ts.stop()


ts = timestat("Dropping Last Columns")
artistData['artist'].drop(["ArtistDescr", "NA14"], axis=1, inplace=True)
ts.stop()


savedir = setDir(basedir, "MusicBrainzData")
savename = setFile(savedir, "{0}.p".format("ArtistDataFrame"))
ts = timestat("Saving Master Artist DataFrame To {0} (~20 sec)".format(savename))
from fileIO import pickleIO
pickleIO().save(idata=artistData['artist'], ifile=savename)
fsize = fileInfo(savename).fsize
print("Master Artist DataFrame is {0}{1}".format(fsize[0], fsize[1]))
ts.stop()

tsArtist.stop()
del artistData

# Master Artist Summary Data

In [3]:
savedir = setDir(basedir, "MusicBrainzData")
io = fileIO()
ts = timestat("Loading Master Data")
masterArtistData = io.get(setFile(savedir, "{0}.p".format("ArtistDataFrame")))
masterReleaseGroupData = io.get(setFile(savedir, "{0}.p".format("ReleaseGroupDataFrame")))
ts.stop()

Current Time is Tue Oct 26, 2021 18:15 for Loading Master Data
Process [Loading Master Data] Took 13.7 Seconds


In [4]:
Nmod = 100

In [5]:
from dbArtistID import artistIDMusicBrainz
mbID = artistIDMusicBrainz()
ts = timestat("Setting My ArtistID From MusicBrainz GID")
masterArtistData["MyArtistID"] = masterArtistData['ArtistGID'].apply(mbID.getArtistID)
ts.stop()

Current Time is Tue Oct 26, 2021 18:16 for Setting My ArtistID From MusicBrainz GID
Process [Setting My ArtistID From MusicBrainz GID] Took 15.7 Seconds


In [6]:
def createReleaseGroupKey(x):
    key = None
    primary   = x['ReleaseGroupPrimaryType']
    secondary = x['ReleaseGroupSecondaryType']
    if isinstance(primary, str) and isinstance(secondary, str):
        key = " + ".join([primary, secondary])
    elif isinstance(primary, str):
        key = primary
    elif isinstance(secondary, str):
        key = secondary
    else:
        key = "Unknown"
    return key
    
ts = timestat("Setting ReleaseGroup Key (~1 min)")
masterReleaseGroupData["ReleaseGroupKey"] = masterReleaseGroupData.apply(createReleaseGroupKey, axis=1)
ts.stop()

Current Time is Tue Oct 26, 2021 18:16 for Setting ReleaseGroup Key (~1 min)
Process [Setting ReleaseGroup Key (~1 min)] Took 57.8 Seconds


In [7]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass, artistDBFileInfoClass
from artistDBBase import artistDBTextClass, artistDBLinkClass
from strUtils import fixName
from dbUtils import utilsDiscogs

def getMediaCounts(media):
    amcc = artistDBMediaCountsClass()

    credittype = "Releases"
    if amcc.counts.get(credittype) == None:
        amcc.counts[credittype] = {}
    for creditsubtype in media.media.keys():
        amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))

    return amcc

savedir = setDir(basedir, "MusicBrainzMetadata")
tsAll = timestat("Creating DB Data")
for m,modVal in enumerate(range(Nmod)):
    
    ts = timestat("Creating ModData Subset")
    artistModData = masterArtistData[masterArtistData["MyArtistID"].apply(lambda x: int(x)%Nmod) == modVal]
    releaseGroupModData = masterReleaseGroupData[masterReleaseGroupData["ArtistID"].isin(artistModData.index)]
    ts.stop()

    modValData = {}
    N = artistModData.shape[0]
    tsMod = timestat("Creating DB Data From {0} Artists For ModVal={1}".format(N,modVal))
    for i,(artistID,artistData) in enumerate(artistModData.iterrows()):
        artistName  = artistData["ArtistName"]
        artistGID   = artistData['ArtistGID']
        url         = "https://musicbrainz.org/artist/{0}".format(artistGID)
        myID        = artistData["MyArtistID"]

        generalData = {}
        generalData["SortName"]   = artistData["ArtistSortName"]
        generalData["Aliases"]    = artistData["Aliases"]
        generalData["Gender"]     = artistData["Gender"]
        generalData["County"]     = artistData["Country"]
        generalData["Formed"]     = artistData["Formed"]
        generalData["Disbanded"]  = artistData["Disbanded"]
        generalData["ArtistType"] = artistData["ArtistType"]
        generalData["ISNI"]       = artistData["ISNICode"]
        generalData = {k: v for k,v in generalData.items() if v is not None}
        generalData = generalData if len(generalData) > 0 else None

        
        ########################################################################
        # Get Releases
        ########################################################################
        artistReleaseGroupData = releaseGroupModData[releaseGroupModData["ArtistID"] == artistID]
        mediaData = {}
        for mediaName,mediaNameData in artistReleaseGroupData.groupby("ReleaseGroupKey"):
            mediaData[mediaName] = []
            for code, releaseGroupInfo in mediaNameData.iterrows():
                album        = releaseGroupInfo['ReleaseGroupName']
                albumURL     = "https://musicbrainz.org/releasegroup/{0}".format(releaseGroupInfo['ReleaseGroupGID'])
                albumArtists = [artistName]
            
                amdc = artistDBMediaDataClass(album=album, url=albumURL, artist=albumArtists, code=code, year=None)
                mediaData[mediaName].append(amdc)
            
            
        artist      = artistDBNameClass(name=artistName, err=None)
        meta        = artistDBMetaClass(title=None, url=url)
        url         = artistDBURLClass(url=url)
        ID          = artistDBIDClass(ID=myID)
        pages       = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        profile     = artistDBProfileClass(general=generalData)
        media       = artistDBMediaClass()
        media.media = mediaData
        mediaCounts = getMediaCounts(media)
        info        = artistDBFileInfoClass(info=None)
        
        modValData[myID] = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, info=info)
        
        if (i+1) % 7500 == 0 or (i+1) == 2500:
            tsMod.update(n=i+1, N=N)
    tsMod.stop()
            
    outdir = setDir(basedir, "MusicBrainzDBData")
    io.save(idata=Series(modValData), ifile=setFile(outdir, "{0}-{1}.p".format(modVal, "DB")))
    tsAll.update(n=m, N=Nmod)
    print("\n")
tsAll.stop()

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Current Time is Tue Oct 26, 2021 18:17 for Creating DB Data
Current Time is Tue Oct 26, 2021 18:17 for Creating ModData Subset
Process [Creating ModData Subset] Took 2.2 Seconds
Current Time is Tue Oct 26, 2021 18:17 for Creating DB Data From 18813 Artists For ModVal=0
2500/18813 : Process [Creating DB Data From 18813 Artists For ModVal=0] Has Run For 4.8 Seconds.  ETA is 31.3 Seconds
7500/18813 : Process [Creating DB Data From 18813 Artists For ModVal=0] Has Run For 12.7 Seconds.  ETA is 19.2 Seconds
15000/18813 : Process [Creating DB Data From 18813 Artists For ModVal=0] Has Run For 23.9 Seconds.  ETA is 6.1 Seconds
Process [Creating DB Data From 18813 Artists For ModVal=0] Took 30.1 Seconds
0/100      : Process [Creating DB Data] Has Run For 33.8 Seconds.  ETA is ? Seconds


Current Time is Tue Oct 26, 2021 18:17 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.6 Seconds
Current Time is Tue Oct 26, 2021 18:18 for Creating DB Data From 18789 Artists For ModVal=1
2

2500/18920 : Process [Creating DB Data From 18920 Artists For ModVal=11] Has Run For 9.3 Seconds.  ETA is 61.1 Seconds
7500/18920 : Process [Creating DB Data From 18920 Artists For ModVal=11] Has Run For 26.7 Seconds.  ETA is 40.7 Seconds
15000/18920 : Process [Creating DB Data From 18920 Artists For ModVal=11] Has Run For 39.9 Seconds.  ETA is 10.4 Seconds
Process [Creating DB Data From 18920 Artists For ModVal=11] Took 46.8 Seconds
11/100     : Process [Creating DB Data] Has Run For 7.8 Minutes.  ETA is 63.1 Minutes


Current Time is Tue Oct 26, 2021 18:25 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.8 Seconds
Current Time is Tue Oct 26, 2021 18:25 for Creating DB Data From 18954 Artists For ModVal=12
2500/18954 : Process [Creating DB Data From 18954 Artists For ModVal=12] Has Run For 6.9 Seconds.  ETA is 45.4 Seconds
7500/18954 : Process [Creating DB Data From 18954 Artists For ModVal=12] Has Run For 17.3 Seconds.  ETA is 26.4 Seconds
15000/18954 : Process [C

7500/18856 : Process [Creating DB Data From 18856 Artists For ModVal=22] Has Run For 12.3 Seconds.  ETA is 18.6 Seconds
15000/18856 : Process [Creating DB Data From 18856 Artists For ModVal=22] Has Run For 22.4 Seconds.  ETA is 5.8 Seconds
Process [Creating DB Data From 18856 Artists For ModVal=22] Took 27.5 Seconds
22/100     : Process [Creating DB Data] Has Run For 14.8 Minutes.  ETA is 52.5 Minutes


Current Time is Tue Oct 26, 2021 18:32 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.6 Seconds
Current Time is Tue Oct 26, 2021 18:32 for Creating DB Data From 18833 Artists For ModVal=23
2500/18833 : Process [Creating DB Data From 18833 Artists For ModVal=23] Has Run For 5.0 Seconds.  ETA is 32.7 Seconds
7500/18833 : Process [Creating DB Data From 18833 Artists For ModVal=23] Has Run For 12.0 Seconds.  ETA is 18.1 Seconds
15000/18833 : Process [Creating DB Data From 18833 Artists For ModVal=23] Has Run For 22.0 Seconds.  ETA is 5.6 Seconds
Process [Creating DB Da

15000/18914 : Process [Creating DB Data From 18914 Artists For ModVal=33] Has Run For 30.7 Seconds.  ETA is 8.0 Seconds
Process [Creating DB Data From 18914 Artists For ModVal=33] Took 37.8 Seconds
33/100     : Process [Creating DB Data] Has Run For 22.0 Minutes.  ETA is 44.7 Minutes


Current Time is Tue Oct 26, 2021 18:39 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.8 Seconds
Current Time is Tue Oct 26, 2021 18:39 for Creating DB Data From 18971 Artists For ModVal=34
2500/18971 : Process [Creating DB Data From 18971 Artists For ModVal=34] Has Run For 6.3 Seconds.  ETA is 41.5 Seconds
7500/18971 : Process [Creating DB Data From 18971 Artists For ModVal=34] Has Run For 15.6 Seconds.  ETA is 23.9 Seconds
15000/18971 : Process [Creating DB Data From 18971 Artists For ModVal=34] Has Run For 29.0 Seconds.  ETA is 7.7 Seconds
Process [Creating DB Data From 18971 Artists For ModVal=34] Took 35.1 Seconds
34/100     : Process [Creating DB Data] Has Run For 22.7 Minutes.

Process [Creating DB Data From 19113 Artists For ModVal=44] Took 38.5 Seconds
44/100     : Process [Creating DB Data] Has Run For 29.3 Minutes.  ETA is 37.3 Minutes


Current Time is Tue Oct 26, 2021 18:46 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.8 Seconds
Current Time is Tue Oct 26, 2021 18:46 for Creating DB Data From 18919 Artists For ModVal=45
2500/18919 : Process [Creating DB Data From 18919 Artists For ModVal=45] Has Run For 6.8 Seconds.  ETA is 44.7 Seconds
7500/18919 : Process [Creating DB Data From 18919 Artists For ModVal=45] Has Run For 16.6 Seconds.  ETA is 25.3 Seconds
15000/18919 : Process [Creating DB Data From 18919 Artists For ModVal=45] Has Run For 30.5 Seconds.  ETA is 8.0 Seconds
Process [Creating DB Data From 18919 Artists For ModVal=45] Took 37.4 Seconds
45/100     : Process [Creating DB Data] Has Run For 30.0 Minutes.  ETA is 36.7 Minutes


Current Time is Tue Oct 26, 2021 18:47 for Creating ModData Subset
Process [Creating ModData Sub

Process [Creating ModData Subset] Took 1.8 Seconds
Current Time is Tue Oct 26, 2021 18:54 for Creating DB Data From 18923 Artists For ModVal=56
2500/18923 : Process [Creating DB Data From 18923 Artists For ModVal=56] Has Run For 6.8 Seconds.  ETA is 44.7 Seconds
7500/18923 : Process [Creating DB Data From 18923 Artists For ModVal=56] Has Run For 16.4 Seconds.  ETA is 25.0 Seconds
15000/18923 : Process [Creating DB Data From 18923 Artists For ModVal=56] Has Run For 30.4 Seconds.  ETA is 8.0 Seconds
Process [Creating DB Data From 18923 Artists For ModVal=56] Took 37.5 Seconds
56/100     : Process [Creating DB Data] Has Run For 37.6 Minutes.  ETA is 29.5 Minutes


Current Time is Tue Oct 26, 2021 18:55 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.8 Seconds
Current Time is Tue Oct 26, 2021 18:55 for Creating DB Data From 18815 Artists For ModVal=57
2500/18815 : Process [Creating DB Data From 18815 Artists For ModVal=57] Has Run For 6.2 Seconds.  ETA is 40.5 Seconds


2500/18770 : Process [Creating DB Data From 18770 Artists For ModVal=67] Has Run For 4.6 Seconds.  ETA is 29.9 Seconds
7500/18770 : Process [Creating DB Data From 18770 Artists For ModVal=67] Has Run For 11.0 Seconds.  ETA is 16.5 Seconds
15000/18770 : Process [Creating DB Data From 18770 Artists For ModVal=67] Has Run For 20.3 Seconds.  ETA is 5.1 Seconds
Process [Creating DB Data From 18770 Artists For ModVal=67] Took 24.7 Seconds
67/100     : Process [Creating DB Data] Has Run For 45.0 Minutes.  ETA is 22.2 Minutes


Current Time is Tue Oct 26, 2021 19:02 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.5 Seconds
Current Time is Tue Oct 26, 2021 19:02 for Creating DB Data From 19052 Artists For ModVal=68
2500/19052 : Process [Creating DB Data From 19052 Artists For ModVal=68] Has Run For 4.3 Seconds.  ETA is 28.5 Seconds
7500/19052 : Process [Creating DB Data From 19052 Artists For ModVal=68] Has Run For 10.7 Seconds.  ETA is 16.5 Seconds
15000/19052 : Process [C

7500/18812 : Process [Creating DB Data From 18812 Artists For ModVal=78] Has Run For 11.1 Seconds.  ETA is 16.7 Seconds
15000/18812 : Process [Creating DB Data From 18812 Artists For ModVal=78] Has Run For 20.9 Seconds.  ETA is 5.3 Seconds
Process [Creating DB Data From 18812 Artists For ModVal=78] Took 25.7 Seconds
78/100     : Process [Creating DB Data] Has Run For 51.5 Minutes.  ETA is 14.5 Minutes


Current Time is Tue Oct 26, 2021 19:08 for Creating ModData Subset
Process [Creating ModData Subset] Took 1.5 Seconds
Current Time is Tue Oct 26, 2021 19:08 for Creating DB Data From 18940 Artists For ModVal=79
2500/18940 : Process [Creating DB Data From 18940 Artists For ModVal=79] Has Run For 4.6 Seconds.  ETA is 30.2 Seconds
7500/18940 : Process [Creating DB Data From 18940 Artists For ModVal=79] Has Run For 11.1 Seconds.  ETA is 16.9 Seconds
15000/18940 : Process [Creating DB Data From 18940 Artists For ModVal=79] Has Run For 21.0 Seconds.  ETA is 5.5 Seconds
Process [Creating DB Da

15000/18844 : Process [Creating DB Data From 18844 Artists For ModVal=89] Has Run For 35.4 Seconds.  ETA is 9.1 Seconds
Process [Creating DB Data From 18844 Artists For ModVal=89] Took 41.9 Seconds
89/100     : Process [Creating DB Data] Has Run For 58.6 Minutes.  ETA is 7.2 Minutes


Current Time is Tue Oct 26, 2021 19:16 for Creating ModData Subset
Process [Creating ModData Subset] Took 2.3 Seconds
Current Time is Tue Oct 26, 2021 19:16 for Creating DB Data From 18944 Artists For ModVal=90
2500/18944 : Process [Creating DB Data From 18944 Artists For ModVal=90] Has Run For 5.9 Seconds.  ETA is 38.8 Seconds
7500/18944 : Process [Creating DB Data From 18944 Artists For ModVal=90] Has Run For 13.4 Seconds.  ETA is 20.4 Seconds
15000/18944 : Process [Creating DB Data From 18944 Artists For ModVal=90] Has Run For 24.8 Seconds.  ETA is 6.5 Seconds
Process [Creating DB Data From 18944 Artists For ModVal=90] Took 29.7 Seconds
90/100     : Process [Creating DB Data] Has Run For 59.2 Minutes. 

# Merge With Known DB

In [12]:
ts = timestat("Merging DBs")
for n,modVal in enumerate(range(100)):
    newDB = Series(io.get("/Volumes/Seagate/DB/MusicBrainzDBData/{0}-DB.p".format(modVal)))
    known = io.get("/Users/tgadfort/dbdiscogs/artists-musicbrainz-db/{0}-DB.p".format(modVal))
    
    toMerge = newDB[~newDB.index.isin(known.index)]
    fullDB = concat([known,toMerge]).sort_index()
    io.save(idata=fullDB, ifile="/Users/tgadfort/dbdiscogs/artists-musicbrainz-db/full/{0}-DB.p".format(modVal))
    ts.update(n=n+1,N=100)
ts.stop()

Current Time is Wed Oct 27, 2021 08:35 for Merging DBs
1/100      : Process [Merging DBs] Has Run For 7.0 Seconds.  ETA is 693.0 Seconds
2/100      : Process [Merging DBs] Has Run For 14.0 Seconds.  ETA is 686.0 Seconds
3/100      : Process [Merging DBs] Has Run For 20.6 Seconds.  ETA is 666.1 Seconds
4/100      : Process [Merging DBs] Has Run For 27.8 Seconds.  ETA is 667.2 Seconds
5/100      : Process [Merging DBs] Has Run For 35.4 Seconds.  ETA is 672.6 Seconds
6/100      : Process [Merging DBs] Has Run For 42.0 Seconds.  ETA is 658.0 Seconds
7/100      : Process [Merging DBs] Has Run For 48.9 Seconds.  ETA is 649.7 Seconds
8/100      : Process [Merging DBs] Has Run For 56.1 Seconds.  ETA is 645.1 Seconds
9/100      : Process [Merging DBs] Has Run For 1.1 Minutes.  ETA is 11.1 Minutes
10/100     : Process [Merging DBs] Has Run For 1.2 Minutes.  ETA is 10.8 Minutes
11/100     : Process [Merging DBs] Has Run For 1.3 Minutes.  ETA is 10.5 Minutes
12/100     : Process [Merging DBs] Has 

In [11]:
known.shape

(8358,)

In [ ]:
tmp['172552485256597266680385033568580864600'].show()

In [ ]:
masterArtistData = artistData['artist'][["ArtistGID", "ArtistName", "ArtistSortName", "Formed", "Disbanded"]].copy(deep=True)
masterArtistNumAlbums = artistIDNumReleaseGroups.join(artistIDNumRelease, how='outer')
masterArtistData = masterArtistData.join(masterArtistNumAlbums)
masterArtistData["NumReleaseGroups"] = masterArtistData["NumReleaseGroups"].fillna(0).apply(int)
masterArtistData["NumReleases"] = masterArtistData["NumReleases"].fillna(0).apply(int)

In [ ]:
masterArtistData

In [ ]:
masterartistNumAlbums

In [ ]:
artistIDNumReleaseGroups.shape

In [ ]:
artistIDNumRelease.shape

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["ArianaGrande"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["BuddyHolly"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["Bono"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["Rupaul"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["U2"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["DMB"]]

In [ ]:
artistData['artist']["NA18"].value_counts()

In [ ]:
artistData['artist']["NA9"].value_counts()

In [ ]:
artistData['artist']['NA12'].unique()

In [ ]:
artistData['artist']['NA5'].unique()

In [ ]:
artistData['artist'][artistData['artist']['ArtistGID'] == '7f347782-eb14-40c3-98e2-17b6e1bfe56c']

In [ ]:
artistData["artist"][artistData["artist"]["ArtistID"] == 502]

In [ ]:
artistData["artist"]['NA10'].unique()

In [ ]:
artistData["artist"][artistData["artist"]["ArtistID"] == 197]

# Artist Lookup

In [ ]:
colnames["l_artist_url"]={0: "ArtistURLLID", 1: "URLGroupID", 2: "ArtistID", 3: "URLID"}
colnames["l_artist_release_group"]={0: "ArtistReleaseGroupLID", 1: "ReleaseGroupGroupID", 2: "ArtistID", 3: "ReleaseGroupID"}
colnames["l_artist_release"]={0: "ArtistReleaseLID", 1: "ReleaseGroupID", 2: "ArtistID", 3: "ReleaseID"}

ts = timestat("Loading Artist Data")
files = glob("mbdump/l_artist_*")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
lookupData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in lookupData.items() if key in colnames} if lookupData is not None else lookupData
print("Keys: {0}".format(lookupData.keys()))
ts.stop()

In [ ]:
files = glob("mbdump/l_artist_release")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}

In [ ]:
lookupData['l_artist_release']["ReleaseGroupID"].nunique()

In [ ]:
lookupData['l_artist_release'][lookupData['l_artist_release']['ArtistID'] == 502]

In [ ]:
key='l_artist_url'
lookupData['l_artist_url'] = lookupData['l_artist_url'][list(colnames[key].keys())].rename(columns=colnames[key])

In [ ]:
print(urlData['url'][urlData['url']["URLName"].eq("https://www.discogs.com/artist/6520")])
print(urlData['url'][urlData['url']["URLName"].eq("https://www.allmusic.com/artist/mn0000219203")])

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"]["URLID"].isin([3017,993955])]

In [ ]:
from pandas import merge
dmbAU = lookupData["l_artist_url"][lookupData["l_artist_url"]['ArtistID'] == 502].copy(deep=True)
u2AU  = lookupData["l_artist_url"][lookupData["l_artist_url"]['ArtistID'] == 197].copy(deep=True)

In [ ]:
dmbURLs = merge(dmbAU, urlData['url'], how='left', on=["URLID"]).copy(deep=True)
u2URLs  = merge(u2AU, urlData['url'], how='left', on=["URLID"]).copy(deep=True)

In [ ]:
dmbURLs["URLDomain"] = dmbURLs["URLName"].apply(lambda x: x.replace("https://", "").replace("http://", "").split('/')[0])

In [ ]:
u2URLs["URLDomain"] = u2URLs["URLName"].apply(lambda x: x.replace("https://", "").replace("http://", "").split('/')[0])

In [ ]:
u2URLs[["NA1", "URLDomain"]].sort_values(by="NA1").T

In [ ]:
dmbURLs[["NA1", "URLDomain"]].sort_values(by="NA1").T

In [ ]:
artistData["artist"][artistData["artist"].eq(8723).any(1)]

In [ ]:
urlData['url'].shape

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"].eq(1025971).any(1)]

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"].eq(2625).any(1)]

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"][2].eq(502).any(1)]

In [ ]:
DMB={AllMusic = 1025971 (c94225e3-2f0c-4c6d-9115-9f268fb7c31b), Discogs = 2625 (7a157b6e-d01d-4248-9995-edb05652c5b2)}

In [ ]:
artistData['artist']

In [ ]:
colnames = {0: "ArtistID", 1: "NA1", 2: "NA2": 3: "NA3"}
lookupData["l_artist_artist"][lookupData["l_artist_artist"].eq(502).any(1)]

In [ ]:
urlData['url'][urlData['url']["URLName"].eq("https://www.discogs.com/artist/6520")]
urlData['url'][urlData['url']["URLName"].eq("https://www.allmusic.com/artist/mn0000219203")]

In [ ]:
502
07e748f1-075e-428d-85dc-ce3be434e906